## Get blog posts

- Extract markdown from Jina blog posts
- Convert to plain text
- Store in one big string to send to GPT later (for generating questions)

In [2]:
from helper import *

In [3]:
# first ten pages of jina.ai/news, excluding docarray/jina release posts (which are all very samey and basic)
urls = [
    "https://jina.ai/news/reader-lm-small-language-models-for-cleaning-and-converting-html-to-markdown",
    "https://jina.ai/news/jina-colbert-v2-multilingual-late-interaction-retriever-for-embedding-and-reranking",
    "https://jina.ai/news/late-chunking-in-long-context-embedding-models",
    "https://jina.ai/news/the-what-and-why-of-text-image-modality-gap-in-clip-models",
    "https://jina.ai/news/by-hoovering-up-the-web-ai-is-poisoning-itself",
    "https://jina.ai/news/what-we-learned-at-icml2024-ft-plag-xrm-tinybenchmark-magiclens-prompt-sketching-etc",
    "https://jina.ai/news/jina-embeddings-and-reranker-on-azure-scalable-business-ready-ai-solutions",
    "https://jina.ai/news/having-it-both-ways-combining-bm25-with-ai-reranking",
    "https://jina.ai/news/smaller-faster-cheaper-jina-rerankers-turbo-and-tiny",
    "https://jina.ai/news/enhancing-search-results-with-jina-ais-reranker-api-in-myscale",
    "https://jina.ai/news/retrieve-jira-tickets-with-jina-reranker-and-haystack-20",
    "https://jina.ai/news/dspy-not-your-average-prompt-engineering",
    "https://jina.ai/news/elevating-youtube-scripts-with-promptperfect-ai-mastery-for-video-content-creators",
    "https://jina.ai/news/next-level-cloud-ai-jina-embeddings-and-rerankers-on-amazon-sagemaker",
    "https://jina.ai/news/click-worthy-content-with-promptperfect-ai-marketing-for-newsletters-and-social-media",
    "https://jina.ai/news/maximizing-search-relevancy-and-rag-accuracy-with-jina-reranker",
    "https://jina.ai/news/revolutionizing-bilingual-text-embeddings-with-multi-task-contrastive-learning",
    "https://jina.ai/news/what-is-colbert-and-late-interaction-and-why-they-matter-in-search",
    "https://jina.ai/news/theres-a-time-and-a-place-unleashing-dynamic-variables-in-promptperfect",
    "https://jina.ai/news/dify-ai-integrates-jina-embeddings-for-rag",
    "https://jina.ai/news/jina-embeddings-v2-and-mongodb-atlas",
    "https://jina.ai/news/scenexplains-json-schema-store-automate-your-alt-text-and-more",
    "https://jina.ai/news/artificial-general-intelligence-is-cursed-and-science-fiction-isnt-helping",
    "https://jina.ai/news/jina-ai-8k-embedding-models-hit-aws-marketplace-for-on-prem-deployment",
    "https://jina.ai/news/scenexplains-ocr-beats-gpt-4v-hands-down-chinese-japanese-korean-arabic-hindi-and-more",
    "https://jina.ai/news/embeddings-in-depth",
    "https://jina.ai/news/look-up-in-the-sky-using-scenexplain-to-classify-land-use-from-satellite-data",
    "https://jina.ai/news/jina-embeddings-2-the-best-solution-for-embedding-long-documents",
    "https://jina.ai/news/case-study-revolutionizing-e-commerce-user-experience-and-streamlining-search-with-scenexplain",
    "https://jina.ai/news/the-1950-2024-text-embeddings-evolution-poster",
    "https://jina.ai/news/words-json-images-scenexplains-new-json-schema-builder",
    "https://jina.ai/news/full-stack-rag-with-jina-embeddings-v2-and-llamaindex",
    "https://jina.ai/news/a-magic-carpet-ride-building-vivid-product-stories-with-scenexplain",
    "https://jina.ai/news/multi-agent-simulations-in-promptperfect-n-heads-are-better-than-one",
    "https://jina.ai/news/a-tale-of-two-worlds-emnlp-2023-at-sentosa",
    "https://jina.ai/news/graph-embedding-101-unraveling-the-magic-of-relational-data",
    "https://jina.ai/news/fine-tuning-gpt-3-5-turbo-a-costly-mirage",
    "https://jina.ai/news/the-boundless-horizon-of-ai-its-not-just-about-the-size",
    "https://jina.ai/news/building-a-streaming-api-for-llama-2-real-time-ai-with-jina-and-docarray",
    "https://jina.ai/news/training-smarter-not-harder-slimming-sentence-embeddings",
    "https://jina.ai/news/beyond-pixels-to-prose-scenexplains-hearth-algorithm-breathes-audible-life-into-images",
    "https://jina.ai/news/whats-new-in-promptperfect-new-optimizer-streaming-bulk-optimization",
    "https://jina.ai/news/scenexplains-image-json-extract-structured-data-images-precision",
    "https://jina.ai/news/embeddings-the-swiss-army-knife-of-ai",
    "https://jina.ai/news/promptperfects-llm-database-blurring-boundaries-search-vs-generation",
    "https://jina.ai/news/distilled-ai-using-large-models-to-teach-smaller-ones",
    "https://jina.ai/news/do-you-truly-need-a-dedicated-vector-store",
    "https://jina.ai/news/how-scenexplain-solves-video-to-text-comprehension",
    "https://jina.ai/news/building-rag-with-jina-ai-and-superduperdb",
    "https://jina.ai/news/precise-rag-with-jina-reranker-and-llamaindex",
    "https://jina.ai/news/build-a-rag-system-with-jina-embeddings-and-qdrant"
]

In [4]:
def read(url):

    url = f"https://r.jina.ai/{url}"

    headers = {
        "X-Return-Format": "text",
        "X-No-Cache": "true",
        "X-Timeout": "1000"
    }

    response = requests.get(url, headers=headers)
    return response.text

In [109]:
import time

# sometimes reader doesn't get the text
def read_with_retry(url, retries=3, delay=2):
    """Attempt to read content from a URL, retrying up to `retries` times if content is empty."""
    attempt = 0
    text = ""
    
    while attempt < retries:
        print(f"Attempt {attempt + 1} for {url}")
        try:
            text = read(url)  # assuming `read(url)` fetches the text
            if text:  # Check if the content is non-empty
                break  # Exit loop if text is successfully fetched
        except Exception as e:
            print(f"Error reading {url}: {e}")
        
        attempt += 1
        if attempt < retries:
            time.sleep(delay)  # Optional: Delay between retries

    return text

In [110]:
docs = []

In [111]:
import requests
import os

for url in urls:
    doc = BlogPost(url=url)
    doc.filename = os.path.basename(url)
    # print(f"Processing {doc.filename}")
    
    doc.text = read_with_retry(url)

    if len(doc.text.strip()) != 0:
        docs.append(doc)

Attempt 1 for https://jina.ai/news/reader-lm-small-language-models-for-cleaning-and-converting-html-to-markdown
Attempt 1 for https://jina.ai/news/jina-colbert-v2-multilingual-late-interaction-retriever-for-embedding-and-reranking
Attempt 1 for https://jina.ai/news/late-chunking-in-long-context-embedding-models
Attempt 1 for https://jina.ai/news/the-what-and-why-of-text-image-modality-gap-in-clip-models
Attempt 1 for https://jina.ai/news/by-hoovering-up-the-web-ai-is-poisoning-itself
Attempt 1 for https://jina.ai/news/what-we-learned-at-icml2024-ft-plag-xrm-tinybenchmark-magiclens-prompt-sketching-etc
Attempt 1 for https://jina.ai/news/jina-embeddings-and-reranker-on-azure-scalable-business-ready-ai-solutions
Attempt 1 for https://jina.ai/news/having-it-both-ways-combining-bm25-with-ai-reranking
Attempt 1 for https://jina.ai/news/smaller-faster-cheaper-jina-rerankers-turbo-and-tiny
Attempt 1 for https://jina.ai/news/enhancing-search-results-with-jina-ais-reranker-api-in-myscale
Attempt

### See how many failed

In [112]:
len(urls) - len(docs)

1

## Save as pickle

In [124]:
with open("docs-populated.pkl", "wb") as file:
    pickle.dump(docs, file)

## Combine into bigger corpora for topic/q/a generation

In [125]:
def split_list(lst, x):
    """Splits the given list into sublists of size x."""
    return [lst[i:i + x] for i in range(0, len(lst), x)]

In [126]:
split_lists = split_list(docs, 20)

In [127]:
def concat_strings(doc_list):
    return '---'.join(doc.text for doc in doc_list)

In [128]:
# split docs into different batches, otherwise exceed GPT-4 turbo context window. Each batch is now under ~100k tokens acc to OpenAI tokenizer
texts = []
for i, lst in enumerate(split_lists, start=1):
    concat_text = concat_strings(lst)
    texts.append(concat_text)

In [129]:
len(split_lists)

3

## Generate topics, questions, answers

In [ ]:
!pip install -q langchain
!pip install -q langchain-openai

In [130]:
system_prompt = """
The user is going to upload several batches of blog posts. your job is to:
- Generate five topics that are covered in the text
- For each topic, generate five deep technical questions which can only be answered by reading the text
- Provide answers to those questions in a manner similar to a RAG system

Ensure your answers are accurate, detailed, and concise.

Provide your output in a JSON structure as follows:

```json
[
  {
    "topic": "topic_name",
    "questions": [
      {
        "question": question,
        "answer": answer
      },
      {
        "question": question,
        "answer": answer
      },
      {
        "question": question,
        "answer": answer
      },
      {
        "question": question,
        "answer": answer
      },
      {
        "question": question,
        "answer": answer
      },
    ]
  },
  {
    "topic": "topic_name",
    "questions": [
      {
        "question": question,
        "answer": answer
      },
      {
        "question": question,
        "answer": answer
      },
      {
        "question": question,
        "answer": answer
      },
      {
        "question": question,
        "answer": answer
      },
      {
        "question": question,
        "answer": answer
      },
    ]
  }
]
```

Return only pure JSON as output. Do not wrap the output in backticks.
"""

In [121]:
OPENAI_API_BASE = "https://api2.road2all.com/v1/"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4o-2024-08-06",
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE,
)

In [122]:
import json

def response_to_json(text):
    # sometimes output is code-fenced
    if text[0] == "`":
        lines = text.split('\n')
        text = '\n'.join(lines[1:-1])

    json_output = json.loads(text)

    return json_output

In [123]:
def generate_questions(texts):
    topics_qa = []
    for i, text in enumerate(texts, start=1):
        print(f"Generating for text [{i}/{len(texts)}]")
        messages = [
        (
            "system",
            system_prompt,
        ),
        ("human", texts[0]),
        ]
        response = llm.invoke(messages).content

        response = response_to_json(response)
        topics_qa.extend(response)
    
    return topics_qa

In [131]:
topics_questions_answers = generate_questions(texts)

Generating for text [1/3]
Generating for text [2/3]
Generating for text [3/3]


In [132]:
len(topics_questions_answers)

15

---

In [133]:
import pickle

with open("topics_questions_answers.pkl", "wb") as file:
    file.write(pickle.dumps(topics_questions_answers))